Notebook Settings
=================

``` ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%run ../../notebooks/setup.py
%matplotlib inline
%config InlineBackend.figure_format = 'png'
```

Imports
=======

``` ipython
import sys
sys.path.insert(0, '../../')

import torch
import gc
import pandas as pd
from time import perf_counter

from src.network import Network
from src.plot_utils import plot_con
from src.decode import decode_bump
from src.utils import clear_cache
```

Helpers
=======

``` ipython
def convert_seconds(seconds):
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    return h, m, s
```

RNN with torch
==============

Single Trial
------------

Here we will run a single simulation with the parameters provided in
**/conf/config<sub>2pop</sub>.yml**.

``` ipython
start = perf_counter()

# We need to define the project root
REPO_ROOT = "/home/leon/models/NeuroFlame/"

# First we create a network with
model = Network('config_2pop.yml', REPO_ROOT)

# then we run the simulation with
output = model()

print('output', output.shape)
# model outputs a tensor of rates of size (N_BATCH, N_STEPS, N_NEURON), so we need to convert it to numpy

rates = output[0].cpu().numpy()
print('rates', rates.shape)

end = perf_counter()
print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))

Ne = model.Na[0].detach().cpu().numpy()
N = model.N_NEURON
```

We can delete the model with

``` ipython
print('gpu memory before del:', torch.cuda.memory_allocated()/100000)
del model
clear_cache()
print('gpu memory after del:', torch.cuda.memory_allocated()/100000)
```

Let's look at the activities of the neurons

``` ipython
fig, ax = plt.subplots(1, 3, figsize=(2*width, height))

r_max = 10

ax[0].imshow(rates.T, aspect='auto', cmap='jet', vmin=0, vmax=r_max, origin='lower')
ax[0].set_ylabel('Neuron #')
ax[0].set_xlabel('Step')

ax[1].plot(rates.mean(-1))
for i in range(10):
    ax[1].plot(rates[..., i], alpha=0.2)

ax[1].set_ylabel('$<Rates>_i$')
ax[1].set_xlabel('Step')
ax[1].set_ylim([0, r_max])

ax[2].hist(rates[-1], density=True, bins='auto')
ax[2].set_xlabel('Density')
ax[2].set_ylabel('Rates')
plt.show()
```

``` ipython
```

Multiple Trials
---------------

### Multiple Initial Conditions

We can run multiple initializations of the network changing
N<sub>BATCH</sub> to the number of initializations that we want.

``` ipython
model = Network('config_2pop.yml', REPO_ROOT, GAIN=5)

model.N_BATCH = 10
rates = model().cpu().numpy()
print('rates', rates.shape)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=(2*width, height))

for i in range(rates.shape[0]):
    ax[0].hist(rates.mean(1)[i], bins=20, density=True)
ax[0].set_ylabel('$<Rates>_i$')
ax[0].set_xlabel('Initialization')

ax[1].plot(rates.mean(-1).T)
ax[1].set_ylabel('$<Rates>_i$')
ax[1].set_xlabel('Step')
plt.show()
```

``` ipython
print(torch.cuda.memory_allocated()/100000)
del model
clear_cache()
print(torch.cuda.memory_allocated()/100000)
```

### Batching Feedforward Inputs

To run some parameter searches, we can easily batch over a different set
of ff inputs Let's see an example where we change the ff inputs to the
excitatory population

1.  The easy way (but memory consuming)

    We create a batch of inputs of size (N<sub>BATCH</sub>,
    N<sub>STEPS</sub>, N<sub>NEURON</sub>)

    ``` ipython
    model = Network('config_2pop.yml', REPO_ROOT)

    ff_list = np.linspace(0, 10, 10)
    ff_inputs = []
    for i in ff_list:
        model.Ja0[:, 0] = i  # here we set the ff input to E to value i in 0 .. 10
        ff_inputs.append(model.init_ff_input())

    ff_inputs = torch.vstack(ff_inputs)
    print('ff_inputs', ff_inputs.shape)
    ```

    Then we pass these inputs to the model

    ``` ipython
    rates = model(ff_inputs).cpu().numpy()
    print(rates.shape)
    ```

    ``` ipython
    fig, ax = plt.subplots(1, 2, figsize=(2*width, height))

    ax[0].plot(rates.mean((1,-1)), '-o')
    ax[0].set_ylabel('$<Rates>_i$')
    ax[0].set_xlabel('FF inputs')

    ax[1].plot(rates.mean(-1).T)  
    ax[1].set_ylabel('$<Rates>_i$')
    ax[1].set_xlabel('Step')
    ax[1].set_ylim([0, 30])
    plt.show()
    ```

    ``` ipython
    print(torch.cuda.memory_allocated()/100000)
    del model
    clear_cache()
    print(torch.cuda.memory_allocated()/100000)
    ```

2.  The hard way (slow but more memory friendly)

    We create a batch of ff inputs that are updated at each time step

    ``` ipython
    model = Network('config_2pop.yml', REPO_ROOT, LIVE_FF_UPDATE=1)

    N_BATCH = 10
    print('original ff_input', model.Ja0.shape)

    new_Ja0 = model.Ja0.repeat((N_BATCH, 1, 1)) 
    print('new ff_input', new_Ja0.shape)

    new_Ja0[:, 0] = torch.linspace(0, 10, N_BATCH, device='cuda').unsqueeze(-1) * model.M0 * torch.sqrt(model.Ka[0])
    print('batched ff_input', new_Ja0[:, 0].squeeze(-1))
    ```

    ``` ipython
    model.N_BATCH = N_BATCH
    model.Ja0 = new_Ja0
    model.LIVE_FF_UPDATE = 1

    start = perf_counter()
    rates = model().cpu().numpy()
    end = perf_counter()
    print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))

    print('rates', rates.shape)
    ```

    ``` ipython
    fig, ax = plt.subplots(1, 2, figsize=(2*width, height))

    ax[0].plot(rates.mean((1,-1)), '-o')
    ax[0].set_ylabel('$<Rates>_i$')
    ax[0].set_xlabel('FF inputs')

    ax[1].plot(rates.mean(-1).T)  
    ax[1].set_ylabel('$<Rates>_i$')
    ax[1].set_xlabel('Step')
    ax[1].set_ylim([0, 30])
    plt.show()
    ```

    ``` ipython
    print(torch.cuda.memory_allocated()/100000)
    del model
    clear_cache()
    print(torch.cuda.memory_allocated()/100000)
    ```

### Batching Reccurent Weights Jab

``` ipython
model = Network('config_2pop.yml', REPO_ROOT, IF_STP=0, DT=0.001, GAIN=0.5, VERBOSE=0, LIVE_FF_UPDATE=1)
```

``` ipython
model.IF_BATCH_J = 1

Jee_list = torch.linspace(0.0, 1.5, 10, device='cuda')  
model.Jab_batch = Jee_list.unsqueeze(-1) * model.Jab[0, 0]
print(model.Jab_batch[:, 0])

model.N_BATCH = model.Jab_batch.shape[0]
model.VERBOSE = 0
```

``` ipython
start = perf_counter()
rates_Jee = model().cpu().detach().numpy()
end = perf_counter()
print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))

print('rates', rates.shape)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

mean_rates = rates_Jee[:,-1].mean(-1)

ax[0].plot(Jee_list.cpu().numpy(), mean_rates)
ax[0].set_xlabel('$J_{EE}$')
ax[0].set_ylabel('$<Rates>_i$')
# ax[0].set_ylim([0, 60])

ax[1].plot(rates_Jee.mean(-1).T)
ax[1].set_xlabel('$J_{EE}$')
ax[1].set_ylabel('Rates')
# ax[1].set_ylim([0, 60])
plt.show()
```

``` ipython
print(torch.cuda.memory_allocated()/100000)
del model
clear_cache()
print(torch.cuda.memory_allocated()/100000)
```

### Batching Feedforward Inputs and Weights

``` ipython
model = Network('config_2pop.yml', REPO_ROOT, IF_STP=0, DT=0.001, GAIN=0.5, LIVE_FF_UPDATE=1)
```

``` ipython
N_BATCH = 10

JEE = torch.linspace(0.0, 5.0, N_BATCH, device='cuda')
JE0 = torch.linspace(0.0, 5.0, N_BATCH, device='cuda')

JEE = JEE.unsqueeze(1).expand(N_BATCH, N_BATCH) 
JEE = JEE.reshape((-1, 1)) * model.Jab[0, 0]
print('Jee', JEE.shape)

JE0 = JE0.unsqueeze(0).expand(N_BATCH, N_BATCH)
JE0 = JE0.reshape((-1, 1))
print('Je0', JE0.shape)

new_Ja0 = model.Ja0.repeat((N_BATCH*N_BATCH, 1, 1)) 

print('Ja0', new_Ja0.shape)
new_Ja0[:,0] = JE0 * torch.sqrt(model.Ka[0]) * model.M0
```

``` ipython
print(JEE[:, 0].reshape(N_BATCH, N_BATCH)[0])
print(JEE[:, 0].reshape(N_BATCH, N_BATCH)[:, 0])
```

``` ipython
print(new_Ja0[..., 0, 0].reshape(N_BATCH, N_BATCH)[0])
print(new_Ja0[..., 0, 0].reshape(N_BATCH, N_BATCH)[:, 0])
```

``` ipython
model.IF_BATCH_J = 1
model.Jab_batch = JEE * model.Jab[0, 0]

model.Ja0 = new_Ja0

model.N_BATCH = model.Jab_batch.shape[0]
model.VERBOSE = 0

start = perf_counter()
rates = model().cpu().detach().numpy()
end = perf_counter()
print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))

print('rates', rates.shape)
```

``` ipython
mean_rates = rates.mean(-1).reshape(N_BATCH, N_BATCH, -1)
print(mean_rates[0, :, -1])
print(mean_rates[:, 0, -1])
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].imshow(mean_rates[..., -1].T, cmap='jet', origin='lower', aspect='auto')
ax[0].set_xlabel('$J_{EE}$')
ax[0].set_ylabel('$J_{E0}$')

ax[1].plot(mean_rates[-1, :, -1]) # over inputs
ax[1].plot(mean_rates[:, -1, -1]) # over Js

ax[1].set_xlabel('$J_{EE}$')
ax[1].set_ylabel('$J_{E0}$')

plt.show()
```

``` ipython
print(torch.cuda.memory_allocated()/100000)
del model
clear_cache()
print(torch.cuda.memory_allocated()/100000)
```